## Tuo tarvittavat kirjastot  
Tuo PyTorch- ja transformers-kirjastot, joita tarvitaan Phi-4-mallin lataamiseen ja käyttämiseen.


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

## Aseta satunnaissiementä
Aseta satunnaissiementä varmistaaksesi toistettavat tulokset eri suorituskerroilla.


In [ ]:
torch.random.manual_seed(0)

## Lataa Phi-4-mini-flash-reasoning-malli ja Tokenizer
Lataa Microsoftin Phi-4-mini-flash-reasoning-malli ja sen vastaava tokenizer Hugging Facesta. Malli ladataan CUDA:lle nopeampaa päättelyä varten.


In [ ]:
model_id = "microsoft/Phi-4-mini-flash-reasoning"
model = AutoModelForCausalLM.from_pretrained(
   model_id,
   device_map="cuda",
   torch_dtype="auto",
   trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

## Valmistele Syöttöviesti
Luo keskusteluviesti, joka sisältää toisen asteen yhtälön matemaattisen ongelman, ja muotoile se mallin keskustelumallin avulla.


In [ ]:
messages = [{
   "role": "user",
   "content": "How to solve 3*x^2+4*x+5=1?"
}]   
inputs = tokenizer.apply_chat_template(
   messages,
   add_generation_prompt=True,
   return_dict=True,
   return_tensors="pt",
)

## Luo Vastaus
Luo vastaus mallista käyttämällä määritettyjä parametreja, kuten lämpötila ja top_p, hallitaksesi satunnaisuutta vastauksessa.


In [ ]:
outputs = model.generate(
   **inputs.to(model.device),
   max_new_tokens=32768,
   temperature=0.6,
   top_p=0.95,
   do_sample=True,
)

## Muunna tulos tekstiksi  
Muunna luodut token-sekvenssit takaisin ihmisen luettavaksi tekstiksi, jättäen alkuperäiset syötetokenit pois, jotta näkyy vain mallin vastaus.


In [ ]:
outputs = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[-1]:])


---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäinen asiakirja sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
